In [10]:
from firebase import firebase
from datetime import datetime
firebase = firebase.FirebaseApplication('https://alert-7c2b1.firebaseio.com/', None)

In [11]:

firebase.put('/alerts/alert1','mic_id',0)
firebase.put('/alerts/alert1','description','safe')
firebase.put('/alerts/alert1','timestamp','--')

'--'

In [12]:
import pickle
import os
from scipy.io import wavfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from python_speech_features import mfcc

In [13]:
class Config:
    def __init__(self, mode='time', nfilt=26, nfeat= 13, nfft=512, rate=16000):
        self.mode = mode
        self.nfilt = nfilt
        self.nfeat = nfeat
        self.nfft = nfft
        self.rate = rate
        self.step = int(rate/10)
        self.model_path = os.path.join('models', mode + '.model')
        self.p_path = os.path.join('pickles', mode + '.p')



In [14]:
def build_predictions(audio_file):
    y_true = []
    fn_prob = {}
    
    print('Extracting features from audio')
    
    rate, wav = wavfile.read(audio_file)
    y_prob = []

    for i in range(0, wav.shape[0]-config.step, config.step):
        sample = wav[i:i+config.step]
        X = mfcc(sample, rate, numcep=config.nfeat, nfilt=config.nfilt, nfft=config.nfft)
        X = (X - config.min)/(config.max - config.min)

        if config.mode == 'conv':
            X = X.reshape(1, X.shape[0], X.shape[1], 1)
        elif config.mode == 'time':
            X = np.expand_dims(X, axis=0)
        elif config.mode == 'convlstm':
            X = X.reshape(1, X.shape[0], X.shape[1], 1)
        y_hat = model.predict(X)
        y_prob.append(y_hat)
        

    fn_prob[audio_file] = np.mean(y_prob, axis=0).flatten()
        
    return fn_prob

In [15]:
classes = ['background', 'glass', 'screams']
p_path = os.path.join('pickles', 'conv.p')

with open(p_path, 'rb') as handle:
    config = pickle.load(handle)
    
model = load_model(config.model_path)

fn_prob = build_predictions('demo.wav')


Extracting features from audio


In [16]:
y_probs = []
y_prob = fn_prob['demo.wav']
y_probs.append(y_prob)
        
y_pred = [classes[np.argmax(y)] for y in y_probs]
print(y_pred)


['screams']


In [17]:
alert_mic = 1001;
alert_desc = y_pred[0]

if alert_desc !='background':
    firebase.put('/alerts/alert1','mic_id',alert_mic)
    firebase.put('/alerts/alert1','description',alert_desc)
    firebase.put('/alerts/alert1','timestamp',datetime.now())